Mobility-Guided-Intelligent-Assistant - Main File

Authors			    :	Kyaw Kyaw Oo<br>
Emails              :   ook1@mcmaster.ca<br>
Student No          :   400551761<br>
Group No			:   Sixth Smart Sense Generation<br>
Started Date        :   Jan 28, 2025<br>
Version             :   1<br>
Description       	: 	Skeleton of Visula Perception and GenAI<br>
Version             :   2 <br>
Description       	: 	Adding Yolo and Utility<br>
Version             :   3 <br>
Description       	: 	Fine Tune Object Detection and Adding Text to Speech then Save<br>
Version             :   4 <br>
Description       	: 	Enhance Object Detection and Added Analysis Report Generation<br>
Version             :   5 <br>
Description       	: 	Added Information Analysis Report<br>
Version             :   6 <br>
Description       	: 	Code Clean and Refactoring if needed<br>
Version             :   7 <br>
Description       	: 	Code Clean and Refactoring if needed<br>
Released Date       :   Apr 7, 2025

CopyRight@2024 by Sixth Smart Sense Generation<br>

In [ ]:
# Install necessary packages

# %pip install pyttsx3
%pip install gTTS -q
%pip install --upgrade torch -q
%pip install gguf>=0.10.0 -q
%pip install transformers -q # if using Hugging Face's transformers
%pip install tqdm -q # For progress bars


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: 0.10.0 not found
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Load basic libraries
import numpy as np

In [ ]:
# load image libraries
from PIL import Image
import cv2 as cv

In [ ]:
# laod Pretrained models
from ultralytics import YOLO

# Visual Perception Module

# Improvement of Object Detection

In [ ]:

def detect_objects_yolov8(frame, model, confidence_threshold=0.5):
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes
    confs = results[0].boxes.conf.cpu().numpy()  # Confidence scores
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)  # Class IDs

    for box, conf, class_id in zip(boxes, confs, class_ids):
        if conf < confidence_threshold:
            continue
        x1, y1, x2, y2 = map(int, box)
        label = f"{model.names[class_id]}: {conf:.2f}"
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return frame

def capture_video_and_save_gif(video_source="on the way.mp4", output_gif="output.gif", duration=0.1):
    model = YOLO("yolov8n.pt")  # Load YOLOv8 model (smallest version)
    
    cap = cv.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error opening video file.")
        return
    
    frames = []
    frame_skip = 2  # Process every 2nd frame
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip != 0:
            frame_count += 1
            continue
        
        # Detect objects using YOLOv8
        processed_frame = detect_objects_yolov8(frame, model)
        
        # Convert BGR to RGB for PIL and store the frame
        pil_frame = Image.fromarray(cv.cvtColor(processed_frame, cv.COLOR_BGR2RGB))
        frames.append(pil_frame)
        
        # Show live detection
        cv.imshow("Live Object Detection", processed_frame)
        cv.waitKey(1)  # Brief delay to update the display

    cap.release()
    cv.destroyAllWindows()
    
    if frames:
        # Save the processed frames as a GIF; duration is in milliseconds
        frames[0].save(output_gif, save_all=True, append_images=frames[1:], duration=int(duration * 1000), loop=0)
        print(f"GIF saved as {output_gif}")
    else:
        print("No frames captured, GIF not saved.")

if __name__ == "__main__":
    capture_video_and_save_gif()


0: 640x640 1 car, 78.7ms
Speed: 2.4ms preprocess, 78.7ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 68.9ms
Speed: 1.5ms preprocess, 68.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 61.8ms
Speed: 1.4ms preprocess, 61.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 65.7ms
Speed: 1.1ms preprocess, 65.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 183.5ms
Speed: 2.8ms preprocess, 183.5ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 82.0ms
Speed: 2.3ms preprocess, 82.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 56.0ms
Speed: 1.4ms preprocess, 56.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 train, 48.5ms
Speed: 1.1ms preprocess, 48.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640

: 

In [ ]:

def detect_objects_yolov8(frame, model, target_classes=None, conf_threshold=0.5):
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
    
    # Filter for target classes if specified
    if target_classes:
        target_classes = [model.names.index(c) for c in target_classes if c in model.names]
        mask = np.isin(class_ids, target_classes)
        boxes = boxes[mask]
        confs = confs[mask]
        class_ids = class_ids[mask]

    # Create class-specific color map
    class_colors = {
        0: (0, 255, 0),    # person - green
        2: (255, 0, 0),    # car - blue
        5: (0, 0, 255),    # bus - red
        7: (255, 255, 0)   # truck - cyan
    }

    # Object counter
    class_counts = {}

    for box, conf, class_id in zip(boxes, confs, class_ids):
        if conf < conf_threshold:
            continue

        x1, y1, x2, y2 = map(int, box)
        class_name = model.names[class_id]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1

        # Get color from predefined map or generate random color
        color = class_colors.get(class_id, tuple(np.random.randint(0, 255, 3).tolist()))

        # Enhanced bounding box
        cv.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        
        # Improved label background
        label = f"{class_name} {conf:.2f}"
        (w, h), _ = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv.rectangle(frame, (x1, y1 - h - 10), (x1 + w, y1), color, -1)
        
        # White text for better contrast
        cv.putText(frame, label, (x1, y1 - 10), 
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Display counts in top-left corner
    count_text = " | ".join([f"{k}: {v}" for k, v in class_counts.items()])
    cv.putText(frame, count_text, (10, 30), 
               cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv.LINE_AA)

    return frame

def capture_video_and_save_gif(video_source="on the way.mp4", 
                              output_gif="output.gif",
                              duration=0.1,
                              target_classes=None,
                              conf_threshold=0.5):
    model = YOLO("yolov8n.pt")
    
    cap = cv.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error opening video file.")
        return
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame = detect_objects_yolov8(
            frame, 
            model,
            target_classes=target_classes,
            conf_threshold=conf_threshold
        )
        
        pil_frame = Image.fromarray(cv.cvtColor(processed_frame, cv.COLOR_BGR2RGB))
        frames.append(pil_frame)
        
        cv.imshow("Enhanced Object Detection", processed_frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()
    
    if frames:
        frames[0].save(output_gif, save_all=True, 
                       append_images=frames[1:], 
                       duration=int(duration * 1000), 
                       loop=0)
        print(f"Enhanced GIF saved as {output_gif}")
    else:
        print("No frames captured.")

if __name__ == "__main__":
    # Example: Only detect vehicles with high confidence
    capture_video_and_save_gif(
        target_classes=["car", "bus", "truck"],
        conf_threshold=0.6
    )

0: 640x640 1 car, 571.6ms
Speed: 11.4ms preprocess, 571.6ms inference, 38.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 589.2ms
Speed: 92.2ms preprocess, 589.2ms inference, 21.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 849.5ms
Speed: 261.3ms preprocess, 849.5ms inference, 29.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 326.9ms
Speed: 5.4ms preprocess, 326.9ms inference, 17.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 199.4ms
Speed: 8.4ms preprocess, 199.4ms inference, 12.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 212.3ms
Speed: 3.8ms preprocess, 212.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 220.5ms
Speed: 4.2ms preprocess, 220.5ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 train, 179.9ms
Speed: 5.5ms preprocess, 179.9ms inference, 5.8ms postprocess per image at

: 

In [ ]:

def detect_objects_yolov8(frame, model, target_classes=None, conf_threshold=0.5):
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
    
    # Filter for target classes if specified
    if target_classes:
        target_classes = [model.names.index(c) for c in target_classes if c in model.names]
        mask = np.isin(class_ids, target_classes)
        boxes = boxes[mask]
        confs = confs[mask]
        class_ids = class_ids[mask]

    # Create class-specific color map
    class_colors = {
        0: (0, 255, 0),    # person - green
        2: (255, 0, 0),    # car - blue
        5: (0, 0, 255),    # bus - red
        7: (255, 255, 0)   # truck - cyan
    }

    # Object counter
    class_counts = {}

    for box, conf, class_id in zip(boxes, confs, class_ids):
        if conf < conf_threshold:
            continue

        x1, y1, x2, y2 = map(int, box)
        class_name = model.names[class_id]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1

        # Get color from predefined map or generate random color
        color = class_colors.get(class_id, tuple(np.random.randint(0, 255, 3).tolist()))

        # Enhanced bounding box
        cv.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        
        # Improved label background
        label = f"{class_name} {conf:.2f}"
        (w, h), _ = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv.rectangle(frame, (x1, y1 - h - 10), (x1 + w, y1), color, -1)
        
        # White text for better contrast
        cv.putText(frame, label, (x1, y1 - 10), 
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Display counts in top-left corner
    count_text = " | ".join([f"{k}: {v}" for k, v in class_counts.items()])
    cv.putText(frame, count_text, (10, 30), 
               cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv.LINE_AA)

    return frame

def capture_video_and_save_gif(video_source="on the way.mp4", 
                              output_gif="output.gif",
                              duration=0.1,
                              target_classes=None,
                              conf_threshold=0.5):
    model = YOLO("yolov8n.pt")
    
    cap = cv.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error opening video file.")
        return
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame = detect_objects_yolov8(
            frame, 
            model,
            target_classes=target_classes,
            conf_threshold=conf_threshold
        )
        
        pil_frame = Image.fromarray(cv.cvtColor(processed_frame, cv.COLOR_BGR2RGB))
        frames.append(pil_frame)
        
        cv.imshow("Enhanced Object Detection", processed_frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()
    
    if frames:
        frames[0].save(output_gif, save_all=True, 
                       append_images=frames[1:], 
                       duration=int(duration * 1000), 
                       loop=0)
        print(f"Enhanced GIF saved as {output_gif}")
    else:
        print("No frames captured.")

if __name__ == "__main__":
    # Example: Only detect vehicles with high confidence
    capture_video_and_save_gif(
        target_classes=["car", "bus", "truck"],
        conf_threshold=0.6
    )


0: 640x640 1 car, 80.0ms
Speed: 2.6ms preprocess, 80.0ms inference, 8.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 61.0ms
Speed: 1.5ms preprocess, 61.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 54.2ms
Speed: 1.0ms preprocess, 54.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 51.5ms
Speed: 1.0ms preprocess, 51.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 156.7ms
Speed: 0.9ms preprocess, 156.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 64.9ms
Speed: 1.2ms preprocess, 64.9ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 45.7ms
Speed: 1.1ms preprocess, 45.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 train, 51.5ms
Speed: 1.2ms preprocess, 51.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640

: 

# Adding Semantic Segementation Approach

In [12]:
import os
from pathlib import Path

main_dir = os.getcwd()
folder_path = Path(main_dir)

# Get parent directory
parent_folder = folder_path.parent
input_folder = parent_folder / "data/input"
output_folder = parent_folder / "data/output"

in_file_name = "On the way.mp4"
out_file_name = "output.gif"

input_file_path = str(input_folder) + "/" + in_file_name
output_file_path = str(output_folder) + "/" + out_file_name
input_file_path, output_file_path

('/Users/kyawkyawoo/Documents/Automation and Smart System/742 Visual Perception/Projects/SEP-742-VIAV/data/input/On the way.mp4',
 '/Users/kyawkyawoo/Documents/Automation and Smart System/742 Visual Perception/Projects/SEP-742-VIAV/data/output/output.gif')

In [ ]:

import numpy as np
import cv2 as cv
from ultralytics import YOLO
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision import models

def apply_segmentation_mask(frame, segmentation_model):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((frame.shape[0], frame.shape[1])),
        transforms.ToTensor(),
    ])
    
    frame_tensor = transform(frame).unsqueeze(0)
    with torch.no_grad():
        output = segmentation_model(frame_tensor)['out'][0]
    mask = output.argmax(0).byte().cpu().numpy()
    colored_mask = cv.applyColorMap(mask * 20, cv.COLORMAP_JET)
    
    return cv.addWeighted(frame, 0.7, colored_mask, 0.3, 0)

def detect_objects_yolov8(frame, model, segmentation_model=None, target_classes=None, conf_threshold=0.5):
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
    
    if segmentation_model:
        frame = apply_segmentation_mask(frame, segmentation_model)
    
    if target_classes:
        target_classes = [model.names.index(c) for c in target_classes if c in model.names]
        mask = np.isin(class_ids, target_classes)
        boxes = boxes[mask]
        confs = confs[mask]
        class_ids = class_ids[mask]
    
    class_colors = {
        0: (0, 255, 0),    # person - green
        2: (255, 0, 0),    # car - blue
        5: (0, 0, 255),    # bus - red
        7: (255, 255, 0)   # truck - cyan
    }
    
    class_counts = {}
    for box, conf, class_id in zip(boxes, confs, class_ids):
        if conf < conf_threshold:
            continue
        
        x1, y1, x2, y2 = map(int, box)
        class_name = model.names[class_id]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        color = class_colors.get(class_id, tuple(np.random.randint(0, 255, 3).tolist()))
        cv.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        
        label = f"{class_name} {conf:.2f}"
        (w, h), _ = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv.rectangle(frame, (x1, y1 - h - 10), (x1 + w, y1), color, -1)
        cv.putText(frame, label, (x1, y1 - 10), 
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    count_text = " | ".join([f"{k}: {v}" for k, v in class_counts.items()])
    cv.putText(frame, count_text, (10, 30), 
               cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv.LINE_AA)
    
    return frame

def capture_video_and_save_gif(video_source=input_file_path, output_gif=output_file_path, duration=0.1, target_classes=None, conf_threshold=0.5):
    try:
        model = YOLO("yolov8n.pt")
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return

    try:
        segmentation_model = models.segmentation.deeplabv3_resnet101(weights=models.segmentation.DeepLabV3_ResNet101_Weights.DEFAULT).eval()
    except Exception as e:
        print(f"Error loading segmentation model: {e}")
        return
    
    cap = cv.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error opening video file.")
        return
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame = detect_objects_yolov8(
            frame, 
            model,
            segmentation_model=segmentation_model,
            target_classes=target_classes,
            conf_threshold=conf_threshold
        )
        
        pil_frame = Image.fromarray(cv.cvtColor(processed_frame, cv.COLOR_BGR2RGB))
        frames.append(pil_frame)
        
        cv.imshow("Enhanced Object Detection", processed_frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv.destroyAllWindows()
    
    if frames:
        frames[0].save(output_gif, save_all=True, 
                       append_images=frames[1:], 
                       duration=int(duration * 1000), 
                       loop=0)
        print(f"Enhanced GIF saved as {output_gif}")
    else:
        print("No frames captured.")

if __name__ == "__main__":
    model = YOLO("yolov8n.pt")
    valid_classes = [c for c in ["car", "bus", "truck", "building"] if c in model.names]
    capture_video_and_save_gif(target_classes=valid_classes, conf_threshold=0.6)



0: 640x640 1 car, 90.9ms
Speed: 3.0ms preprocess, 90.9ms inference, 6.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 63.6ms
Speed: 1.6ms preprocess, 63.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 58.7ms
Speed: 1.1ms preprocess, 58.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 47.4ms
Speed: 1.2ms preprocess, 47.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 60.5ms
Speed: 1.7ms preprocess, 60.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 50.6ms
Speed: 1.0ms preprocess, 50.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 53.7ms
Speed: 1.2ms preprocess, 53.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 train, 48.0ms
Speed: 1.2ms preprocess, 48.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


: 

# Text Information Assistance Module

In [1]:
# Suppress warnings and load environment settings
import warnings

import base64

from PIL import Image

In [2]:
# load Custom libraries
from utility import disp_image, load_env
from utility import llama32

In [3]:

warnings.filterwarnings('ignore')
load_env()

In [7]:
import os
from pathlib import Path

main_dir = os.getcwd()
folder_path = Path(main_dir)

# Get parent directory
parent_folder = folder_path.parent
output_folder = parent_folder / "data/output"

in_gif_name = "output.gif"
out_translation_name = "detection_translation.pdf"

input_gif_path = str(output_folder) + "/" + in_gif_name
output_translatione_path = str(output_folder) + "/" + out_translation_name
input_gif_path, output_translatione_path

('/Users/kyawkyawoo/Documents/Automation and Smart System/742 Visual Perception/Projects/SEP-742-VIAV/data/output/output.gif',
 '/Users/kyawkyawoo/Documents/Automation and Smart System/742 Visual Perception/Projects/SEP-742-VIAV/data/output/detection_translation.pdf')

In [9]:

def llama32pi(prompt, image_url, model_size=11):
    """
    Send a prompt along with an image URL to the Llama32 model for analysis.
    
    Args:
    - prompt (str): The user query/question.
    - image_url (str): The base64 image string.
    - model_size (int): The size of the model to use (default 11).
    
    Returns:
    - result: Response from the Llama32 model.
    """
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": image_url}}
            ]
        },
    ]
    return llama32(messages, model_size)

def encode_image(image_path):
    """
    Convert an image into a base64 encoded string for API usage.
    
    Args:
    - image_path (str): The path of the image to encode.
    
    Returns:
    - base64_encoded_image (str): The base64 encoded image.
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def gif_to_images(gif_path, output_folder="output_frames"):
    """
    Extract frames from a GIF and save them as individual images.
    
    Args:
    - gif_path (str): Path to the GIF file.
    - output_folder (str): Folder where individual frames will be saved.
    
    Returns:
    - frame_paths (list): List of paths to the saved individual frame images.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    with Image.open(gif_path) as gif:
        num_frames = gif.n_frames
        print(f"Total frames: {num_frames}")
        
        frame_paths = []
        for frame_num in range(num_frames):
            gif.seek(frame_num)  # Set the pointer to the frame
            frame = gif.copy()  # Copy the current frame
            
            # Save the frame as a separate image
            frame_path = f"{output_folder}/frame_{frame_num+1}.png"
            frame.save(frame_path)
            frame_paths.append(frame_path)
            print(f"Frame {frame_num+1} saved as {frame_path}")
        
        return frame_paths

def get_vehicle_analysis(image_path):
    """
    Provide a detailed analysis of the vehicle's surroundings, sensors, and environment.
    
    Args:
    - image_path (str): The path to the image of the vehicle.
    
    Returns:
    - analysis_results (dict): Dictionary of analysis results for different questions.
    """
    base64_image = encode_image(image_path)

    # List of questions with descriptions of different vehicle scenarios
    questions = {
        "road_conditions": "Analyze the image of the autonomous vehicle and describe the road conditions, the surrounding environment, and any potential hazards present. What could the vehicle’s sensors be detecting in this scenario?",
        "obstacles_nearby": "Based on the image of the autonomous vehicle on the road, identify any nearby obstacles, other vehicles, or pedestrians. How might the vehicle’s system react to maintain safety and follow traffic rules?",
        "road_signs": "In this image, how might the vehicle’s AI system interpret the road signs, lane markings, and traffic signals visible in the scene? What algorithms or sensor data might be used to navigate through this environment?",
        "vehicle_interaction": "If this autonomous vehicle were to interact with another car in the image, how could it safely navigate the situation? What signals or systems would be involved in the interaction, such as lane changes or speed adjustments?",
        "sensor_analysis": "From the image of the autonomous vehicle’s exterior, what kind of sensors (LIDAR, radar, cameras, etc.) might be present, and how would each of these contribute to the vehicle’s decision-making process in this scenario?",
        "collision_scenario": "In this image, there is a potential collision scenario involving pedestrians and other vehicles. How might an autonomous vehicle’s ethical decision-making algorithm handle this dilemma to minimize harm?",
        "navigation_behavior": "Looking at the image of the autonomous vehicle on a highway, describe the vehicle’s navigation behavior. How does it adjust its speed, lane positioning, or distance from other vehicles based on real-time data?",
        "safety_risks": "Based on the image of the autonomous vehicle, assess the current driving environment for potential risks to the driver's safety. Identify any obstacles, road conditions, nearby vehicles, or pedestrians that the vehicle's sensors might detect. How should the vehicle’s system respond to ensure the driver’s safety, including speed adjustments, lane positioning, or emergency maneuvers?",
        "driving_conditions": "Based on the image of the autonomous vehicle, evaluate the current driving environment, taking into account road conditions, weather factors (such as rain, fog, or snow), traffic complexity, and nearby pedestrians or obstacles. Identify potential risks to the driver's safety and describe how the vehicle’s sensors would detect and assess these hazards. How should the vehicle’s system adjust its speed, lane positioning, or initiate emergency maneuvers to ensure the driver’s safety in these conditions?"
    }

    analysis_results = {}

    # Loop through questions and get the responses
    for key, question in questions.items():
        image_url = f"data:image/jpeg;base64,{base64_image}"
        analysis_results[key] = llama32pi(question, image_url)

    return analysis_results

def display_results(analysis_results):
    """
    Display the analysis results in a readable format.
    
    Args:
    - analysis_results (dict): Dictionary of analysis results for different questions.
    """
    for scenario, result in analysis_results.items():
        print(f"** {scenario.replace('_', ' ').title()} **")
        print(result)
        print("\n" + "-"*50 + "\n")


global analysis_results

# Main functionality for loading the GIF, extracting frames, and processing each frame
if __name__ == "__main__":
    gif_path = input_gif_path   # Replace with the actual path to the GIF file

    frames = gif_to_images(gif_path)  # Extract frames from the GIF
    
    # Process each extracted frame
    for frame in frames:
        print(f"Analyzing frame: {frame}")
        analysis_results = get_vehicle_analysis(frame)
        display_results(analysis_results)

Total frames: 22
Frame 1 saved as output_frames/frame_1.png
Frame 2 saved as output_frames/frame_2.png
Frame 3 saved as output_frames/frame_3.png
Frame 4 saved as output_frames/frame_4.png
Frame 5 saved as output_frames/frame_5.png
Frame 6 saved as output_frames/frame_6.png
Frame 7 saved as output_frames/frame_7.png
Frame 8 saved as output_frames/frame_8.png
Frame 9 saved as output_frames/frame_9.png
Frame 10 saved as output_frames/frame_10.png
Frame 11 saved as output_frames/frame_11.png
Frame 12 saved as output_frames/frame_12.png
Frame 13 saved as output_frames/frame_13.png
Frame 14 saved as output_frames/frame_14.png
Frame 15 saved as output_frames/frame_15.png
Frame 16 saved as output_frames/frame_16.png
Frame 17 saved as output_frames/frame_17.png
Frame 18 saved as output_frames/frame_18.png
Frame 19 saved as output_frames/frame_19.png
Frame 20 saved as output_frames/frame_20.png
Frame 21 saved as output_frames/frame_21.png
Frame 22 saved as output_frames/frame_22.png
Analyzing f

In [ ]:
# Validate the results
# analysis_results

{'road_conditions': 'The image shows a snowy road with a large pile of snow in the foreground, and a building in the background. The road is covered in snow, with tire tracks visible in the foreground. The building in the background appears to be a university, with a sign that reads "McMaster University" in white letters.\n\nThe autonomous vehicle\'s sensors are likely detecting the following:\n\n* The road conditions: The vehicle\'s sensors may be detecting the snow-covered road, the tire tracks, and the surrounding environment.\n* The surrounding environment: The vehicle\'s sensors may be detecting the building, the trees, and the sky.\n* Potential hazards: The vehicle\'s sensors may be detecting potential hazards such as ice, snowdrifts, and obstacles on the road.\n\nThe vehicle\'s sensors may be detecting the following specific features:\n\n* Snow depth: The vehicle\'s sensors may be detecting the depth of the snow on the road, which could affect the vehicle\'s traction and stabili

Overall, the image suggests that the autonomous vehicle is navigating through a challenging winter environment, and its sensors and systems are working to ensure the driver's safety.


# Speech Information Assistance Module

In [11]:

from gtts import gTTS
import os

# Prepare the text to be converted to speech
text = ""

# Iterate through the dictionary and produce speech for each text entry
for key, value in analysis_results.items():
    print(f"** {key.replace('_', ' ').title()} **")
    print(value)
    print("\n" + "-"*50 + "\n")
    
    # Add the key-value pair to the text for speech synthesis
    text += f"The {key.replace('_', ' ')} is {value}. "

# Convert the text to speech
tts = gTTS(text=text, lang='en')

# Save the speech to a file
tts.save(str(output_folder) + "output.mp3")

# Play the speech
os.system(str(output_folder) + "start output.mp3")  # This works on Windows; for Linux/m

** Road Conditions **
The image depicts a snowy scene with a road, trees, and a building in the background. The road is covered in snow, and there are tire tracks visible on it. The trees are bare, and the sky is gray. In the distance, a large building can be seen, with a sign that reads "McMaster University" in white letters.

Based on the image, it appears that the autonomous vehicle is navigating through a winter environment with snow-covered roads and trees. The vehicle's sensors may be detecting the following:

* Snow and ice on the road: The vehicle's sensors may be detecting the presence of snow and ice on the road, which could affect its traction and stability.
* Tire tracks: The vehicle's sensors may be detecting the tire tracks on the road, which could provide information about the road's surface and the presence of other vehicles.
* Trees: The vehicle's sensors may be detecting the trees in the background, which could provide information about the road's surroundings and pot

sh: /Users/kyawkyawoo/Documents/Automation: No such file or directory


32512

In [13]:
# Save as PDf for guided instruction 
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from datetime import datetime

def save_analysis_to_pdf(analysis_results, filename):
    """Save analysis results to a formatted PDF file"""
    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    flowables = []
    
    # Add title
    title_style = styles["Title"]
    flowables.append(Paragraph("Guided Information Analysis Report", title_style))
    flowables.append(Paragraph(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}", styles["Normal"]))
    flowables.append(Spacer(1, 24))
    
    # Add analysis content
    for key, value in analysis_results.items():
        # Section header
        section_title = key.replace('_', ' ').title()
        flowables.append(Paragraph(section_title, styles["Heading2"]))
        
        # Analysis text
        formatted_text = value.replace('\n', '<br/>')
        flowables.append(Paragraph(formatted_text, styles["BodyText"]))
        flowables.append(Spacer(1, 12))
    
    # Build PDF
    doc.build(flowables)
    print(f"PDF report saved to {filename}")

# Usage (after generating analysis_results)
save_analysis_to_pdf(analysis_results, str(output_folder) + "guided_information_analysis_report.pdf")

PDF report saved to /Users/kyawkyawoo/Documents/Automation and Smart System/742 Visual Perception/Projects/SEP-742-VIAV/data/outputguided_information_analysis_report.pdf


# Report Generation Module

Performance Metrics:

Frame processing time and rate

Detection counts and class diversity

API call success/failure rates

Memory and CPU utilization

TTS conversion efficiency

Error Handling:

Graceful failure handling for all stages

Error tracking for individual frame analysis

Resource monitoring to prevent system overload

Evaluation Report:

Comprehensive system resource tracking

Processing time breakdown

Success/failure statistics

Detection performance metrics

Optimizations:

Memory usage tracking

CPU utilization monitoring

Batch processing metrics

Error rate calculations

Empty text detection

Video file verification

In [3]:
%pip install ultralytics opencv-python Pillow reportlab psutil gtts -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# EEvaluaiton Analysis Report
import cv2
import time
import os
import base64
import psutil
from datetime import datetime
from PIL import Image
from ultralytics import YOLO
from gtts import gTTS
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image as ReportLabImage
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch

class VideoAnalysisSystem:
    def __init__(self):
        self.metrics = {
            'video_processing': {},
            'object_detection': {},
            'frame_analysis': {},
            'tts_conversion': {},
            'system_resources': {}
        }
        self.model = YOLO("yolov8n.pt")
        self.analysis_results = {}

    def detect_objects(self, frame, conf_threshold=0.5):
        """Perform object detection with YOLOv8 and metrics collection"""
        start_time = time.time()
        try:
            results = self.model(frame)
            boxes = results[0].boxes.xyxy.cpu().numpy()
            confs = results[0].boxes.conf.cpu().numpy()
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

            detections = []
            for box, conf, class_id in zip(boxes, confs, class_ids):
                if conf < conf_threshold:
                    continue
                
                x1, y1, x2, y2 = map(int, box)
                label = f"{self.model.names[class_id]} {conf:.2f}"
                
                # Green detection elements
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                detections.append(self.model.names[class_id])

            return frame, detections, time.time() - start_time

        except Exception as e:
            print(f"Detection error: {str(e)}")
            return frame, [], 0

    def process_video(self, video_path, conf_threshold=0.5):
        """Process video and generate analysis"""
        cap = cv2.VideoCapture(video_path)
        frames = []
        detections = []
        start_mem = psutil.virtual_memory().used
        start_time = time.time()

        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                processed_frame, frame_dets, _ = self.detect_objects(frame, conf_threshold)
                frames.append(Image.fromarray(cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)))
                detections.extend(frame_dets)

                cv2.imshow("Analysis Preview", processed_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            # Save processed GIF
            if frames:
                frames[0].save(str(output_folder / "output_analysis.gif"), save_all=True, 
                              append_images=frames[1:], duration=100, loop=0)

            # Collect metrics
            self.metrics['video_processing'] = {
                'total_frames': len(frames),
                'processing_time': time.time() - start_time,
                'memory_usage': psutil.virtual_memory().used - start_mem
            }

            self.metrics['object_detection'] = {
                'total_detections': len(detections),
                'unique_classes': len(set(detections)),
                'detection_rate': len(detections)/len(frames) if frames else 0
            }

        finally:
            cap.release()
            cv2.destroyAllWindows()

    def generate_analysis_report(self, output_file="analysis_report.pdf"):
        """Generate comprehensive PDF report"""
        doc = SimpleDocTemplate(output_file, pagesize=letter)
        styles = getSampleStyleSheet()
        elements = []

        # Title Section
        title_style = ParagraphStyle(
            'Title', parent=styles['Title'],
            fontSize=18, alignment=1, spaceAfter=14
        )
        elements.append(Paragraph("Autonomous Vehicle Analysis Report", title_style))
        elements.append(Paragraph(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}", styles['Normal']))
        elements.append(Spacer(1, 0.4*inch))

        # System Overview
        elements.append(Paragraph("System Performance", styles['Heading2']))
        sys_table = Table([
            ['Metric', 'Value'],
            ['Total Processing Time', f"{self.metrics['video_processing']['processing_time']:.2f}s"],
            ['Memory Usage', f"{self.metrics['video_processing']['memory_usage']/1e6:.2f}MB"],
            ['CPU Usage', f"{psutil.cpu_percent()}%"]
        ])
        sys_table.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#4F81BD')),
            ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
            ('GRID', (0,0), (-1,-1), 1, colors.black)
        ]))
        elements.append(sys_table)
        elements.append(Spacer(1, 0.3*inch))

        # Detection Analysis
        elements.append(Paragraph("Object Detection Summary", styles['Heading2']))
        det_table = Table([
            ['Total Detections', self.metrics['object_detection']['total_detections']],
            ['Unique Classes', self.metrics['object_detection']['unique_classes']],
            ['Detection Rate', f"{self.metrics['object_detection']['detection_rate']:.2f}/frame"]
        ])
        det_table.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#4F81BD')),
            ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
            ('GRID', (0,0), (-1,-1), 1, colors.black)
        ]))
        elements.append(det_table)
        elements.append(Spacer(1, 0.3*inch))

        # Sample Frame
        elements.append(Paragraph("Sample Analysis Frame", styles['Heading2']))
        elements.append(ReportLabImage(gif_path, width=4*inch, height=3*inch))
        
        doc.build(elements)
        print(f"Report saved to {output_file}")

if __name__ == "__main__":
    analyzer = VideoAnalysisSystem()
    input = input_file_path
    analyzer.process_video(input)
    analyzer.generate_analysis_report()


0: 640x640 1 car, 101.8ms
Speed: 2.9ms preprocess, 101.8ms inference, 5.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 83.3ms
Speed: 1.9ms preprocess, 83.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 cars, 73.6ms
Speed: 2.0ms preprocess, 73.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 219.5ms
Speed: 1.1ms preprocess, 219.5ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 cars, 83.9ms
Speed: 2.4ms preprocess, 83.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 82.2ms
Speed: 1.1ms preprocess, 82.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 cars, 73.1ms
Speed: 0.9ms preprocess, 73.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 train, 66.9ms
Speed: 0.8ms preprocess, 66.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 6

: 

# Well Done!!! 
# For the Mobility Guided Intelligent Information Assistance of Implementation and Report